<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://github.com/gee-community/geemap/tree/master/tutorials/ImageCollection/02_image_collection_metadata.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_parent"  href="https://nbviewer.jupyter.org/github/gee-community/geemap/blob/master/tutorials/ImageCollection/02_image_collection_metadata.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/gee-community/geemap/blob/master/tutorials/ImageCollection/02_image_collection_metadata.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# ImageCollection Information and Metadata
As with Images, there are a variety of ways to get information about an `ImageCollection`. The collection can be printed directly to the console, but the console printout is limited to 5000 elements. Collections larger than 5000 images will need to be filtered before printing. Printing a large collection will be correspondingly slower. The following example shows various ways of getting information about image collections programmatically:

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/gee-community/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/gee-community/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.foliumap`](https://github.com/gee-community/geemap/blob/master/geemap/foliumap.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print("geemap package not installed. Installing ...")
    subprocess.check_call(["python", "-m", "pip", "install", "geemap"])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.foliumap as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    # ee.Initialize()
    ee.Initialize(project='satellite-example')

## Create an interactive map 
The default basemap is `Google Satellite`. [Additional basemaps](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py#L13) can be added using the `Map.add_basemap()` function. 

In [2]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap("ROADMAP")  # Add Google Map
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

## Add Earth Engine Python script 

In [4]:
# Load a Landsat 8 ImageCollection for a single path-row.
collection = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
    .filter(ee.Filter.eq("WRS_PATH", 44))
    .filter(ee.Filter.eq("WRS_ROW", 34))
    .filterDate("2014-03-01", "2014-08-01")
)
print("Collection: ", collection.getInfo())

Collection:  {'type': 'ImageCollection', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B8', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B9', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B10', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B11', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'QA_PIXEL', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}}, {'id': 'QA_RADSAT', 'data_type': {'type': 'PixelType', 

In [5]:
# Get the number of images.
count = collection.size()
print("Count: ", count.getInfo())

Count:  9


In [6]:
# Get the date range of images in the collection.
range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print(
    "Date range: ",
    ee.Date(range.get("min")).getInfo(),
    ee.Date(range.get("max")).getInfo(),
)

Date range:  {'type': 'Date', 'value': 1395168392053} {'type': 'Date', 'value': 1406227557219}


In [7]:
# Change the date format.
range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
start_date = ee.Date(range.get("min")).format("YYYY-MM-dd")
end_date = ee.Date(range.get("max")).format("YYYY-MM-dd")
print("Date range: ", start_date.getInfo(), end_date.getInfo())

Date range:  2014-03-18 2014-07-24


In [8]:
# Get statistics for a property of the images in the collection.
sunStats = collection.aggregate_stats("SUN_ELEVATION")
print("Sun elevation statistics: ", sunStats.getInfo())

Sun elevation statistics:  {'max': 67.10252727, 'mean': 61.01998433666667, 'min': 46.47106522, 'sample_sd': 7.251804446695445, 'sample_var': 52.58866773311183, 'sum': 549.17985903, 'sum_sq': 33931.6557378883, 'total_count': 9, 'total_sd': 6.837066800129478, 'total_var': 46.74548242943274, 'valid_count': 9, 'weight_sum': 9, 'weighted_sum': 549.17985903}


In [9]:
# Get statistics for a property of the images in the collection.
clouds = collection.aggregate_array("CLOUD_COVER")
print("Cloud covers: ", clouds.getInfo())

Cloud covers:  [0.06, 28.1, 12.9, 24.25, 35.42, 35.71, 33.05, 39.98, 0.32]


In [10]:
# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(collection.sort("CLOUD_COVER").first())
print("Least cloudy image: ", image.propertyNames().getInfo())
print("\nCloud cover: ", image.get("CLOUD_COVER").getInfo())

Least cloudy image:  ['system:version', 'system:id', 'RADIANCE_MULT_BAND_5', 'RADIANCE_MULT_BAND_6', 'RADIANCE_MULT_BAND_3', 'RADIANCE_MULT_BAND_4', 'RADIANCE_MULT_BAND_1', 'RADIANCE_MULT_BAND_2', 'WRS_TYPE', 'K2_CONSTANT_BAND_11', 'K2_CONSTANT_BAND_10', 'system:footprint', 'REFLECTIVE_SAMPLES', 'SUN_AZIMUTH', 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION', 'DATE_ACQUIRED', 'ELLIPSOID', 'STATION_ID', 'RESAMPLING_OPTION', 'ORIENTATION', 'WRS_ROW', 'RADIANCE_MULT_BAND_9', 'TARGET_WRS_ROW', 'RADIANCE_MULT_BAND_7', 'RADIANCE_MULT_BAND_8', 'IMAGE_QUALITY_TIRS', 'TRUNCATION_OLI', 'CLOUD_COVER', 'GEOMETRIC_RMSE_VERIFY', 'COLLECTION_CATEGORY', 'GRID_CELL_SIZE_REFLECTIVE', 'CLOUD_COVER_LAND', 'GEOMETRIC_RMSE_MODEL', 'COLLECTION_NUMBER', 'IMAGE_QUALITY_OLI', 'LANDSAT_SCENE_ID', 'WRS_PATH', 'PANCHROMATIC_SAMPLES', 'PANCHROMATIC_LINES', 'GEOMETRIC_RMSE_MODEL_Y', 'REFLECTIVE_LINES', 'GEOMETRIC_RMSE_MODEL_X', 'system:asset_size', 'system:index', 'DATA_SOURCE_ELEVATION', 'REFLECTANCE_ADD_BAND_1', 'REFLECT

In [11]:
# Limit the collection to the 10 most recent images.
recent = collection.sort("system:time_start", False).limit(10)
images = recent.aggregate_array("system:index").getInfo()
print("Recent images:")
for image in images:
    print(image)

Recent images:
LC08_044034_20140724
LC08_044034_20140708
LC08_044034_20140622
LC08_044034_20140606
LC08_044034_20140521
LC08_044034_20140505
LC08_044034_20140419
LC08_044034_20140403
LC08_044034_20140318


## Display Earth Engine data layers 

In [12]:
Map.addLayerControl()
Map

Map(bottom=1851.0, center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position…